In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import PreferenceModeling
import pyomo
import Rollup
import time
import random
from sklearn import linear_model
import statsmodels.formula.api as smf
import pickle

import seaborn as sns
sns.set(color_codes=True)

pd.options.display.max_columns = 75
pd.options.display.max_rows = 100

path = "../Capstone Data/JDA-Plan/"

In [6]:
texas_df = pd.read_csv(path + 'Texas_sales.csv')
texas_df = texas_df[~texas_df.duplicated(['RESETNAME','rtlr_party_id','UPC'])]
texas_df = texas_df[texas_df["RESETNAME"] == "2017 Spring"]
texas_df["Brand_Package"] = texas_df['Brand'] + " " + texas_df['Package']

demo = pd.read_csv(path + 'demo_TX.rpt')

## Read in data from tree modeling

In [7]:
train = pd.read_csv(path + 'train_df.csv')
test = pd.read_csv(path + 'test_df.csv')
julia_df = pd.concat([train, test])

In [8]:
train.head()

,rtlr_party_id,aggr_id,productTotal,storeTotal,volShare,nSKUs,meanPrice,productTotal_1,storeTotal_1,valueShare,meanPrice_1,productTotal_2,storeTotal_2,movementShare,meanPrice_2,x,cya01v001,cyec17v001,cypopdens,pycypopgrw,xcya04t21p,xcya04t65p,xcya08v001,xcya08v002,xcya08v003,xcya08v004,xcya08v005,xcya08v006,xcya08v007,xcyd04v003,xcyea06v001,xcymale,xmosbpd18,retailer_party_id,retailer_facings,Grocery,Drug,...,s127,s128,s129,s130,s131,s132,s133,s134,s135,s136,s137,s138,s139,s140,s141,s142,s143,s144,s145,s146,s147,s148,s149,s150,s151,s152,s153,s154,s155,s156,s157,s158,s159,s160,s161,s162,leaf
0,5646527,BrandPackageCategory_Bud Light small,51.428571,8665.906571,0.005935,1,0.102584,5.275760,768.545332,0.006865,7.386065,5.275760,768.545332,0.006865,0.714286,671,1245.8,53258.6,660.576,1.041,65.534,11.151,70.918,3.261,1.237,0.276,0.051,21.817,2.439,4.944,64.023,49.485,0.0,5646527.0,124,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,5646527,BrandPackageCategory_Corona Extra medium,22.854857,8665.906571,0.002637,1,0.110194,2.518479,768.545332,0.003277,15.868006,2.518479,768.545332,0.003277,0.158714,671,1245.8,53258.6,660.576,1.041,65.534,11.151,70.918,3.261,1.237,0.276,0.051,21.817,2.439,4.944,64.023,49.485,0.0,5646527.0,124,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,5646527,BrandPackageCategory_Corona Extra small,30.857143,8665.906571,0.003561,1,0.122155,3.769360,768.545332,0.004905,8.795174,3.769360,768.545332,0.004905,0.428571,671,1245.8,53258.6,660.576,1.041,65.534,11.151,70.918,3.261,1.237,0.276,0.051,21.817,2.439,4.944,64.023,49.485,0.0,5646527.0,124,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,5646527,BrandPackageCategory_Corona Light medium,0.000000,8665.906571,0.000000,1,0.105282,0.000000,768.545332,0.000000,15.160587,0.000000,768.545332,0.000000,0.000000,671,1245.8,53258.6,660.576,1.041,65.534,11.151,70.918,3.261,1.237,0.276,0.051,21.817,2.439,4.944,64.023,49.485,0.0,5646527.0,124,0,0,...,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,5646527,BrandPackageCategory_Michelob Ultra large,377.125714,8665.906571,0.043518,1,0.084200,31.753993,768.545332,0.041317,30.312007,31.753993,768.545332,0.041317,1.047571,671,1245.8,53258.6,660.576,1.041,65.534,11.151,70.918,3.261,1.237,0.276,0.051,21.817,2.439,4.944,64.023,49.485,0.0,5646527.0,124,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [10]:
def upcToBrandPackage(prod):
    if prod[0:4] == 'UPC_':
        return(texas_df[texas_df["UPC"].astype(str) == prod[4:]].head(1)[['Brand_Package']].values[0][0])
    else:
        return(prod)
    
upcBrandPackageMapping = {upc:upcToBrandPackage(upc) for upc in julia_df.aggr_id.unique()}
upcBrandPackageMapping['no_purchase'] = 'no_purchase'

## Optimize each leaf separately and report results

#### Train models (only do if not loading pickled results)

In [ ]:
# Get all products in the leaf and generate preferences
products = set(julia_df.aggr_id.unique())
products.add('no_purchase')
prefs = PreferenceModeling.generateRandomPreferences(products, num_start_cols = 200)

In [ ]:
# Find prices in each leaf
prices = dict()
# start by trying to get median prices in that leaf
for leaf in julia_df['leaf'].unique():
    juliaDFLeaf = julia_df[julia_df['leaf'] == leaf]
    prices[leaf] = {prod:juliaDFLeaf[juliaDFLeaf['aggr_id'] == prod]['meanPrice_1'].median() for prod in products}
    assert all(prices[leaf].values()) > 0
    prices[leaf]['no_purchase'] = 0

# if we can't get prices from just the chain, use data from all stores at that chain in the state
for leaf in julia_df['leaf'].unique():
    for prod in products:
        if np.isnan(prices[leaf][prod]):
            prices[leaf][prod] = julia_df[julia_df['aggr_id'] == prod]['meanPrice_1'].median()

# Check that we have all real prices
assert all([all(map(np.isnan, prices[leaf].values())) for leaf in julia_df['leaf'].unique()]) == False

Find the products in a leaf, generate preferences, and optimize

In [ ]:
# Pregenerate the optimization inputs for computational efficiency. Only do this if running the model multiple times on the same leaf
optimizationInputs = dict()
start = time.time()
for leaf in julia_df.leaf.unique():
    print leaf
    optimizationInputs[leaf] = PreferenceModeling.generateVeliborPreferenceModelInputs(julia_df[julia_df['leaf'] == leaf], 
                                                                                       'aggr_id', 'valueShare', 'rtlr_party_id')
print "Cell took %.1f seconds to process" % (time.time() - start)

In [ ]:
# Optimize assortment
start = time.time()
optimal = dict()
for leaf in julia_df.leaf.unique():
    print leaf
    optimal[leaf] = PreferenceModeling.findSingleOptimalAssortment(julia_df[julia_df['leaf'] == leaf], productCol = 'aggr_id', shareCol = 'valueShare', 
                                storeCol = 'rtlr_party_id', priceCol = 'meanPrice', useVelibor = True, prefs = prefs,
                                prices = prices[leaf], optIn = optimizationInputs[leaf])
    print 'The MAE of the preference model is %.4f' % optimal[leaf]['mae']
print "Cell took %.1f seconds to process" % (time.time() - start)

####  Save Model outputs

In [ ]:
filename = path + 'optimizationInputs'
outfile = open(filename,'wb')
pickle.dump(optimizationInputs,outfile)
outfile.close()

In [ ]:
filename = path + 'preferenceModelOutputs'
outfile = open(filename,'wb')
pickle.dump(optimal,outfile)
outfile.close()

## Load model outputs and do reporting

In [11]:
infile = open(path + 'optimizationInputs','rb')
optimizationInputs = pickle.load(infile)
infile.close()

In [ ]:
infile = open(path + 'preferenceModelOutputs','rb')
optimal = pickle.load(infile)
infile.close()

In [ ]:
# Set the leaf you want to look at in evaluation
lf = 4

In [ ]:
print 'The MAE of the preference model is %.4f' % optimal[lf]['mae']

In [ ]:
print 'The most likely consumer types are:'
PreferenceModeling.showTopConsumerTypes(optimal[lf]["prefs"], optimal[lf]['lambda'], mapping = upcBrandPackageMapping, numTypes = 5)

In [ ]:
temp = [upcBrandPackageMapping[k] for (k,v) in optimal[lf]['assortment'].items()]
temp.sort()
print 'The optimal assortment with {0} out of {1} products is:'.format(len(temp),len(optimal[leaf]["products"]))
print
PreferenceModeling.prettyPrintList(temp)

In [ ]:
print 'The predicted shares for the  optimal assortment are'
optimalSharesDf = PreferenceModeling.predictShares(optimal[lf]['assortment'], optimal[lf]['prefs'], optimal[lf]['lambda'], upcBrandPackageMapping)
sns.barplot(y = 'Product', x = 'Share', data = optimalSharesDf)

In [ ]:
tempDf = julia_df[julia_df['leaf'] == lf]
tempStore = tempDf.rtlr_party_id.iloc[0]
tempAssortment = PreferenceModeling.getRealStoreAssortment(tempDf, 'rtlr_party_id', tempStore, 'aggr_id')
print 'The predicted shares for store %d are' % tempStore
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(8, 14)
sns.barplot(y = 'Product', x = 'Share', 
            data = PreferenceModeling.predictShares(tempAssortment, optimal[lf]['prefs'], optimal[lf]['lambda'], upcBrandPackageMapping))

#### Segment breakdown

In [ ]:
productSegmentMapping = pd.read_csv(path + "productSegmentMapping.csv")
productSegmentMapping = productSegmentMapping.append(pd.DataFrame([['No Purchase', 'no_purchase']], columns=['IRI_Sub_Segment','aggr_id']))
# groups = {3:"Grocery",4:"Drug",5:"Convenience"}
optimalSharesDf = pd.DataFrame()
for leaf in julia_df.leaf.unique():
    temp = PreferenceModeling.predictShares(optimal[leaf]['assortment'], optimal[leaf]['prefs'], optimal[leaf]['lambda'])
    temp = temp.merge(productSegmentMapping, left_on = "Product", right_on = "aggr_id", how = "inner")
    temp['leaf'] = leaf
    print leaf
    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(11, 8.5)
    temp.groupby('IRI_Sub_Segment').Share.sum().sort_values().plot.barh()
    plt.show()
    optimalSharesDf = pd.concat([optimalSharesDf,temp])
optimalSharesDf = optimalSharesDf.groupby(['leaf','IRI_Sub_Segment']).Share.sum().reset_index()
optimalSharesDf = optimalSharesDf.sort_values(by = ['leaf', 'Share'], ascending = False)

In [ ]:
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)
sns.barplot(y = 'IRI_Sub_Segment', x = 'Share', hue = 'leaf', data = optimalSharesDf)

In [ ]:
temp = [upcBrandPackageMapping[k] for k in  products]
temp.sort()
PreferenceModeling.prettyPrintList(temp)

#### Evaluate the preference model performance on different products (Fix by passing V through)

In [ ]:
Amat, vmat = PreferenceModeling.convertToMatrix(optimal[lf]["A"], optIn["v"])
assert Amat.shape[0] == vmat.shape[0]

In [ ]:
np.sum(np.abs(np.dot(Amat,optimal[lf]['lambda'].values()) - vmat))/np.shape(Amat)[0]

In [ ]:
def convertToDf(A, v):
    num_rows = len(A[0].keys())
    num_cols = len(A.keys())
    A_matrix = np.zeros((num_rows, num_cols))
    v_matrix =  np.zeros(num_rows)
    for col_counter, col_key in enumerate(A):
        for row_counter, row_key in enumerate(A[0]):
            A_matrix[row_counter, col_counter] = A[col_key][row_key]
    
    for row_counter, row_key in enumerate(A[0]):
         v_matrix[row_counter] = v[row_key][0]
            
    rows = pd.DataFrame(A[0].keys(), columns = ['rtlr_party_id','aggr_id'])
    ADf = pd.concat([rows,pd.DataFrame(A_matrix)], axis = 1)
    vDf = pd.concat([rows,pd.DataFrame(v_matrix, columns = ['share'])], axis = 1)

    return(ADf, vDf)

In [ ]:
ADf, vDf = convertToDf(optimal[lf]["A"], optIn["v"])

In [ ]:
np.sum(np.abs(np.dot(ADf.iloc[:,2:],optimal[lf]['lambda'].values()) - vDf['share']))/np.shape(Amat)[0]

In [ ]:
vDf['predShare'] = np.dot(ADf.iloc[:,2:],optimal[lf]['lambda'].values())

In [ ]:
vDf['absDiff'] = np.abs(vDf['predShare'] - vDf['share'])
vDf['percentDiff'] = np.abs(vDf['predShare'] - vDf['share']) / vDf['share']

In [ ]:
def stdGrp(grp):
    grp['std'] = grp['share'].std()
    return(grp)
def meanGrp(grp):
    grp['mean'] = grp['share'].mean()
    return(grp)
vDf = vDf.groupby('aggr_id').apply(stdGrp)
vDf = vDf.groupby('aggr_id').apply(meanGrp)
vDf['absDiffPercentSTD'] = vDf['absDiff'] / vDf['std']
vDf['meanModelAbsDiff'] = np.abs(vDf['mean'] - vDf['share'])
vDf['modelRatio'] = vDf['absDiff'] / vDf['meanModelAbsDiff']

In [ ]:
vDf.groupby('aggr_id').absDiffPercentSTD.mean().median()

#### To do below (finish optimality gaps)

In [ ]:
pricesDf = pd.DataFrame({upcBrandPackageMapping[k]:v for (k,v) in optimal['prices'].items()}.items(), columns = ['Product', 'Price'])

In [ ]:
def findOptimalityGaps(optimalAssortment, prefs, lam, df, storeCol, productCol, prices, mapping =  'Idenity'):
    '''
    Calculate difference in expected value of current store assortments compared to optimal assortment
    Args:
        optimalAssortment (dict or set): 
    '''
    gaps = dict()
    
    if mapping == 'Identity':
        pricesDf = pd.DataFrame(prices.items(), columns = ['Product', 'Price'])
    else:
        pricesDf = pd.DataFrame({mapping[k]:v for (k,v) in prices.items()}.items(), columns = ['Product', 'Price'])
        
    optimalSharesDf = PreferenceModeling.predictShares(optimalAssortment, prefs, lam, mapping)
    optimalSharesDf = optimalSharesDf.merge(pricesDf, on = 'Product', how = 'inner')
    optEV = np.sum(optimalSharesDf['Share'] * optimalSharesDf['Price'])
    
    for store in df[storeCol].unique():
        storeAssortment = PreferenceModeling.getRealStoreAssortment(df, storeCol, store, productCol)
        storeShareDf = PreferenceModeling.predictShares(storeAssortment, prefs, lam, mapping)
        storeShareDf = storeShareDf.merge(pricesDf, on = 'Product', how = 'inner')
        storeEV = np.sum(storeShareDf['Share'] * storeShareDf['Price'])
        gaps[store] = (optEV - storeEV) / storeEV
    
    gapsDf = pd.DataFrame(gaps.items(), columns = ['rtlr_party_id', 'gap']).sort_values(by = 'gap', ascending = False)
    return(gapsDf)

In [ ]:
findOptimalityGaps(optimal['assortment'], optimal['prefs'], optimal['lambda'], julia_df[julia_df['leaf'] == 4], 
                   'rtlr_party_id', 'aggr_id', optimal['prices'], mapping =  upcBrandPackageMapping)

In [ ]:
optimal = evaluateAssortment(inAssortment, prefs, lam, pricesDf, upcMapping = upcBrandPackageMapping)
gaps = dict()
for store in optIn['stores']:
    tempAssortment = set(julia_df_leaf[julia_df_leaf['rtlr_party_id'] == store].aggr_id.values).union(['no_purchase'])
    tempAssortment = {item:1 for item in tempAssortment}
    storeOut = evaluateAssortment(tempAssortment, prefs, lam, pricesDf, upcMapping = upcBrandPackageMapping)
    gaps[store] = (optimal['ev'] - storeOut['ev'])/storeOut['ev']
gaps = pd.DataFrame(gaps.items(), columns = ['rtlr_party_id', 'gap']).sort_values(by = 'gap', ascending = False)

In [ ]:
gaps.head()

In [ ]:
sns.boxplot(gaps.gap)

## Assortment for chain

We first need to run the preference model on all leaves to get lambdas. Then we calculate prices for a given chain in each leaf. Finally, we optimize/ Steps:
1. Generate preference model inputs for all leaves (done above)
2. Get the set of products available in total (done above)
3. Generate preference rankings (done above)
4. Generate A matrices for each leaf (done above)
5. Run preference model in each leaf (done above)
6. Get prices for the chain in each leaf for each product
7. Get weights for each leaf

In [ ]:
texas_df.groupby(['channel']).CHAINNAME.value_counts()

In [ ]:
prices = dict()
chain = "MURPHY USA"

stripesDf = (texas_df.loc[texas_df['CHAINNAME'] == chain,
              ['rtlr_party_id','CHAINNAME', 'dp_nbr']].drop_duplicates()
             .merge(julia_df, on ='rtlr_party_id', how = 'inner'))
stripesDf = stripesDf[stripesDf.dp_nbr == stripesDf.dp_nbr.iloc[0]]
# start by trying to get median prices in that leaf from just the chain
for leaf in julia_df['leaf'].unique():
    stripesDfLeaf = stripesDf[stripesDf['leaf'] == leaf]
    prices[leaf] = {prod:stripesDfLeaf[stripesDfLeaf['aggr_id'] == prod]['meanPrice_1'].median() for prod in products}
    assert all(prices[leaf].values()) > 0
    prices[leaf]['no_purchase'] = 0

# if we can't get prices from just the chain, use data from all stores at that chain in the state
stripesDfAll = (texas_df.loc[texas_df['CHAINNAME'] == chain,
              ['rtlr_party_id','CHAINNAME', 'dp_nbr']].drop_duplicates()
             .merge(julia_df, on ='rtlr_party_id', how = 'inner'))
for leaf in julia_df['leaf'].unique():
    for prod in products:
        if np.isnan(prices[leaf][prod]):
            prices[leaf][prod] = stripesDfAll[stripesDfAll['aggr_id'] == prod]['meanPrice_1'].median()
            


# finally, just use all data in the leaf
for leaf in julia_df['leaf'].unique():
    for prod in products:
        if np.isnan(prices[leaf][prod]):
            prices[leaf][prod] = julia_df[julia_df['aggr_id'] == prod]['meanPrice_1'].median()
            
# Check that we have all real prices
assert all([all(map(np.isnan, prices[leaf].values())) for leaf in julia_df['leaf'].unique()]) == False

In [ ]:
texas_df[texas_df['dp_nbr'] == stripesDf.dp_nbr.iloc[0]].rtlr_party_id.nunique()

In [ ]:
clusterWeights = dict(stripesDf.leaf.value_counts(normalize=True))
leaves = stripesDf['leaf'].unique()
lam = {leaf:optimal[leaf]['lambda'] for leaf in leaves}

In [ ]:
clusterWeights

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

def optimizeChainDpAssortment(products, prefs, prices, lam, weights, leaves, currentAssortment = None, numChangesFromCurrent = None,
                             disallowedProducts = [], holdAssortmentSizeConstant = False):
    '''
    Find the optimal assortment.
    Args:
        products (list): list of products for the assortment
        prefs (Dict): Dictionary of preference rankings used to generate the A matrix
        prices (Dict): Dictionary of prices (values) associated with each product (key)
        lam (numpy array): Array of coefficients from the preference model or lasso corresponding to weights on the 
                           columns of A
    Returns:
        assortment (Dict): Dictionary with products for keys and 1 or 0 in the value indicating whether a product is in
                           the assortment.
    '''
    
    if numChangesFromCurrent != None:
        assert numChangesFromCurrent >= 0, "changes must be non-negative"
        assert currentAssortment != None, "you must provide the current assortment to specify number of allowed changes"
    
    m = ConcreteModel()
    
    num_products = len(products)
    num_rankings = len(prefs.keys())
    
    # Variables
    m.x = Var(products, within=Binary)
    ySet = [(product, ranking) for product in products for ranking in prefs.keys()]
    assert len(ySet) == num_products * num_rankings
    m.y = Var(ySet, within=NonNegativeReals)

    # Constraints
    def oneChoiceRule(m,ranking):
        return(sum(m.y[(prod, rank)] for (prod, rank) in ySet if rank == ranking) == 1)
    m.oneChoice = Constraint(range(num_rankings), rule = oneChoiceRule)

    def chooseOnlyIfInAssortmentRule(m, product, ranking):
        return(m.y[(product,ranking)] <= m.x[product])
    m.chooseOnlyIfInAssortment = Constraint(products, prefs.keys(), rule = chooseOnlyIfInAssortmentRule)

    def getLessPrefferedProducts(product, ranking):
        indx = prefs[ranking].index(product)
        return(prefs[ranking][(indx+1):])
    def lessPreferredRule(m, product, ranking):
        lessPreffered = getLessPrefferedProducts(product, ranking)
        return(sum(m.y[(prod, ranking)] for prod in lessPreffered) <= 1 - m.x[product])
    m.lessPreffered = Constraint(products, prefs.keys(), rule = lessPreferredRule)
    
    m.alwaysAllowNoPurchase = Constraint(expr = m.x['no_purchase'] == 1)
    
    if numChangesFromCurrent != None:
        currentAssortmentSize = len(currentAssortment)
        m.currentLowerBound = Constraint(expr = sum(m.x[prod] for prod in currentAssortment) >= (currentAssortmentSize - numChangesFromCurrent))
        possibleNewProducts = set(products).difference(set(currentAssortment))
        m.currentUpperBound = Constraint(expr = sum(m.x[prod] for prod in possibleNewProducts) <= numChangesFromCurrent)
    
    def disallowedProductsRule(m, product):
        return(m.x[product] == 0)
    m.disallowedProducts = Constraint(disallowedProducts, rule = disallowedProductsRule)
    
    if holdAssortmentSizeConstant == True:
        assert currentAssortment != None, "you must provide the current assortment to keep the size constant" 
        m.constantAssortmentSize = Constraint(expr = sum(m.x[prod] for prod in products) == len(currentAssortment))
    
    # Objective
    m.obj = Objective(sense = maximize, expr = sum(weights[leaf] * sum(sum(prices[leaf][prod] * lam[leaf][rank] * m.y[(prod, rank)] 
                                                    for prod in products) for rank in prefs.keys())for leaf in leaves))
    
    # Solve model
    solver = SolverFactory('gurobi')
    results = solver.solve(m)
    
    # Returns
    assortment = {prod:m.x[prod].value for prod in products}
    return(assortment)


In [ ]:
currentAssortment = set(stripesDf.aggr_id.value_counts().where(lambda x : x >= 89 * 0.8).dropna().index).union(set(['no_purchase']))

In [ ]:
temp = julia_df.merge(texas_df.loc[texas_df['crctd_city_nm'] == "DALLAS",['rtlr_party_id','crctd_city_nm']], on  = 'rtlr_party_id', how = 'inner')
dallasProducts = temp[temp['crctd_city_nm'] == "DALLAS"].aggr_id.unique()

In [ ]:
dallasProducts.head()

In [ ]:
stripesAssortment = optimizeChainDpAssortment(products = products, prefs = prefs, prices = prices, lam = lam,
                                              weights = clusterWeights, leaves = leaves, currentAssortment = currentAssortment,
                                              numChangesFromCurrent = 1, disallowedProducts = [])

In [ ]:
temp = set([k for (k,v) in stripesAssortment.items() if v > 0]).difference(currentAssortment)
print "The model recommends adding the following packages:"
print
PreferenceModeling.prettyPrintList([upcBrandPackageMapping[x] for x in temp])

In [ ]:
temp = currentAssortment.difference(set([k for (k,v) in stripesAssortment.items() if v > 0]))
print "The model recommends removing the following packages:"
print
PreferenceModeling.prettyPrintList([upcBrandPackageMapping[x] for x in temp])

In [ ]:
temp = [upcBrandPackageMapping[k] for (k,v) in stripesAssortment.items() if v > 0]
temp.sort()
print 'The model recommends having the following {0} out of {1} products:'.format(len(temp),len(products))
print
PreferenceModeling.prettyPrintList(temp)

In [ ]:
temp = set(stripesDf.aggr_id.unique()).difference(set([k for (k,v) in stripesAssortment.items() if v > 0]))
dfChart = stripesDf[stripesDf['aggr_id'].isin(temp)].aggr_id.value_counts().where(lambda x : x >= 89 * 0.8).dropna()
yChart = [upcBrandPackageMapping[x] for x in dfChart.index]
yChart.sort()

print 'The following {0} products are in at least 80% of stores but the model recommends removing them'.format(len(yChart))
print
PreferenceModeling.prettyPrintList(yChart)

In [ ]:
percent = 0.25
prodsInMostStores = stripesDf.aggr_id.value_counts().where(lambda x : x >= 89 * percent).dropna().index
temp2 = set(set([k for (k,v) in stripesAssortment.items() if v > 0])).difference(set(prodsInMostStores)).difference(set(['no_purchase']))
temp2 = [upcBrandPackageMapping[x] for x in temp2]
temp2.sort()

print 'The following {0} products are recommended by the model but are not in at least {1}% of stores'.format(len(temp2), percent * 100)
print
PreferenceModeling.prettyPrintList(temp2)

In [ ]:
optimalSharesDf = pd.DataFrame()
inAssortment = [k for (k,v) in stripesAssortment.items() if v > 0]
for leaf in clusterWeights.keys():
    temp = PreferenceModeling.predictShares(inAssortment, optimal[leaf]['prefs'], optimal[leaf]['lambda'])
    temp = temp.merge(productSegmentMapping, left_on = "Product", right_on = "aggr_id", how = "inner")
    temp['leaf'] = leaf
    optimalSharesDf = pd.concat([optimalSharesDf,temp])

In [ ]:
optimalSharesDf['leafWeight'] = optimalSharesDf.leaf.apply(lambda x: clusterWeights[x])
optimalSharesDf['weightedShare'] = optimalSharesDf['Share'] * optimalSharesDf['leafWeight']
optimalSharesDf = optimalSharesDf[~(optimalSharesDf["Product"] == 'no_purchase')]
optimalSharesDf['scaledShare'] = optimalSharesDf['weightedShare'] / optimalSharesDf['weightedShare'].sum()

In [ ]:
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11, 8.5)
optimalSharesDf.groupby('IRI_Sub_Segment').weightedShare.sum().sort_values().plot.barh()

#### Analysis of assortment performance

In [ ]:
accts = pd.read_csv(path + 'accts_texas2.rpt', error_bad_lines=False)

In [ ]:
sales = pd.read_csv(path + "yoy_sales.csv")
sales = sales[sales['reset17'] == '2017 Spring']
sales = sales[np.abs(sales.yoy_growth-sales.yoy_growth.mean()) <= (3*sales.yoy_growth.std())]
sales = sales[np.abs(sales['yoy_growth']) < 0.4]

In [ ]:
texas_df['movementPerDay'] = texas_df['UNITMOVEMENT'] / texas_df['MOVEMENTPERIODUSED']
averageMovementDf = texas_df.groupby('rtlr_party_id').movementPerDay.sum().reset_index()

In [ ]:
jaccardDf = dict()

for leaf in julia_df.leaf.unique():
    tempDf = julia_df[julia_df['leaf'] == leaf]
    inAssortment = optimal[leaf]['assortment']
    for rtlr in tempDf.rtlr_party_id.unique():
        jaccardDf[rtlr] = (float(len(set(tempDf[tempDf['rtlr_party_id'] == rtlr].aggr_id).intersection(inAssortment))) /
                             float(len(set(tempDf[tempDf['rtlr_party_id'] == rtlr].aggr_id).union(inAssortment))))
    
jaccardDf = pd.DataFrame(jaccardDf.items(), columns = ['rtlr_party_id', 'similarity'])
jaccardDf = jaccardDf.sort_values(by = 'similarity', ascending = False)
jaccardDf = jaccardDf.merge(averageMovementDf, on = 'rtlr_party_id', how = 'inner')
jaccardDf = jaccardDf.merge(accts[['rtlr_party_id', 'channel','segment']], on = 'rtlr_party_id', how = 'inner')
jaccardDf = jaccardDf.merge(sales, left_on='rtlr_party_id', right_on = 'retailer_party_id', how = 'inner')

In [ ]:
jaccardDf.yoy_growth.median()

In [ ]:
mod = smf.ols(formula='yoy_growth ~ similarity + channel', data=jaccardDf)
res = mod.fit()
print(res.summary())

# Old code for generating preference model data

In [ ]:
family_agg_df, family_agg_dict = Rollup.rollup(texas_df,cutoff = 0.6, column = "Category", AB_only = False, other_only = False)

In [ ]:
family_agg_df.agg_UPC.nunique()

In [ ]:
austin_df = texas_df[(texas_df['crctd_city_nm'] == "AUSTIN") & (texas_df['RESETNAME'] == "2017 Spring")]
convenience_df = texas_df[texas_df['channel'] == "CONVENIENCE"]
cluster_df = texas_df[texas_df['segment'] == "Med Hispanic Non-Urban"]

In [ ]:
import random
accounts = family_agg_df.WHOLESALER_CUSTOMER_PARTY_ID.unique()
train_num = int(len(accounts)*0.75) # 0.75 in training set, 0.25 in validation set
train_accounts = random.sample(accounts, train_num)
train = family_agg_df[family_agg_df.WHOLESALER_CUSTOMER_PARTY_ID.isin(train_accounts)]
val = family_agg_df[-family_agg_df.WHOLESALER_CUSTOMER_PARTY_ID.isin(train_accounts)]

In [ ]:
mnlDf_train = PreferenceModeling.generateShareData(train, 'agg_UPC', 'total_volume_oz_per_day', 'categoryShare')
mnlDf_val = PreferenceModeling.generateShareData(val, 'agg_UPC', 'total_volume_oz_per_day', 'categoryShare')

#### MNL

In [ ]:
res = PreferenceModeling.fitMNL(mnlDf_train,'categoryShare','agg_UPC','rtlr_party_id')

In [ ]:
coefs = PreferenceModeling.stripStatsModelVariableInformation(res,'Category','rtlr_party_id')

In [ ]:
preds_test = PreferenceModeling.predictMNL(mnlDf_val,coefs,'Category','rtlr_party_id')
preds_train = PreferenceModeling.predictMNL(mnlDf_train,coefs,'Category','rtlr_party_id')

In [ ]:
mnl_train_absolute_error = np.abs(preds_train['estimated_share'] - preds_train['categoryShare'])
mnl_val_absolute_error = np.abs(preds_test['estimated_share'] - preds_test['categoryShare'])

mnl_val_percent_error = mnl_val_absolute_error / preds_test['categoryShare']

In [ ]:
print("train mape:")
print np.sum(mnl_train_absolute_error / preds_train['categoryShare']) / preds_train.shape[0]
print("train mae:")
print np.sum(mnl_train_absolute_error) / preds_train.shape[0]
print
print("test mape:")
print np.sum(mnl_val_absolute_error / preds_test['categoryShare']) / preds_test.shape[0]
print("test mae:")
print np.sum(mnl_val_absolute_error) / preds_test.shape[0]

#### Velibor

In [ ]:
optIn = PreferenceModeling.generateVeliborPreferenceModelInputs(mnlDf_train,'agg_UPC','categoryShare', 'rtlr_party_id')
evalIn = PreferenceModeling.generateVeliborPreferenceModelInputs(mnlDf_val,'agg_UPC','categoryShare', 'rtlr_party_id')

In [ ]:
result = PreferenceModeling.solveModel(optIn["v"], optIn["assortments"], optIn["stores"], optIn["products"], max_iterations = 5, num_start_cols = 100)

In [ ]:
veliborEval = PreferenceModeling.evaluateVeliborModel(evalIn, result["rankings"], result["master"]["lambda"])
veliborEval

In [ ]:
A = PreferenceModeling.createTestA(evalIn, result["rankings"])
absolute_errors = PreferenceModeling.veliborAbsoluteErrors(A, result["master"]["lambda"], evalIn["v"])

In [ ]:
keys = A[0].keys()
percent_errors = {key:absolute_errors[key]/(evalIn["v"][key][0]) for key in keys}

In [ ]:
print max(absolute_errors.values())
print min(absolute_errors.values())

In [ ]:
print max(percent_errors.values())
print min(percent_errors.values())

In [ ]:
print np.max(mnl_val_absolute_error)
print np.min(mnl_val_absolute_error)

In [ ]:
print np.max(mnl_val_percent_error)
print np.min(mnl_val_percent_error)

In [ ]:
sns.distplot(percent_errors.values())

In [ ]:
sns.distplot(mnl_val_percent_error)

In [ ]:
sns.boxplot(percent_errors.values())

In [ ]:
sns.boxplot(mnl_val_percent_error)

In [ ]:
temp = mnlDf_val.groupby('Category').categoryShare.describe()
temp['normalized_range'] = (temp['max'] - temp['min'])/temp['mean']
temp

In [ ]:
temp = pd.DataFrame(index = percent_errors.keys())
temp['percent_error'] = percent_errors.values()
temp = temp.reset_index()
def firstTup(tup):
    return(tup[0])
def sndTup(tup):
    return(tup[1])
temp['store'] = temp['index'].apply(firstTup)
temp['product'] = temp['index'].apply(sndTup)
temp = temp.drop('index',axis=1)

temp_pivot = temp.groupby('product').percent_error.describe()
temp_pivot['mape_contribution'] = ((temp_pivot['mean'] * temp_pivot['count'])/temp_pivot['count'].sum())/veliborEval['mape']
temp_pivot = temp_pivot.sort_values('mape_contribution',ascending = False)
print (temp_pivot['mean'] * temp_pivot['count']).sum()/temp_pivot['count'].sum()
temp_pivot

In [ ]:
temp_store = temp.groupby('store').percent_error.describe()
temp_store['mape_contribution'] = ((temp_store['mean'] * temp_store['count'])/temp_store['count'].sum())/veliborEval['mape']
temp_store = temp_store.reset_index()
temp_store= temp_store.merge(accts, left_on = 'store', right_on = 'rtlr_party_id',
                                         how = 'inner')
temp_store = temp_store.sort_values('mape_contribution',ascending = False)
temp_store.segment = temp_store.segment.fillna("none")
print (temp_store['mean'] * temp_store['count']).sum() / temp_store['count'].sum() - veliborEval['mape']

In [ ]:
temp_store.head()

In [ ]:
segment_pivot = temp_store.groupby('segment')['mean'].describe()
segment_pivot['mape_contribution'] = ((segment_pivot['mean'] * segment_pivot['count'])/segment_pivot['count'].sum())/veliborEval['mape']
segment_pivot = segment_pivot.reset_index()
segment_pivot = segment_pivot.sort_values('mape_contribution',ascending = False)
segment_pivot

In [ ]:
print (segment_pivot['mean'] * segment_pivot['count']).sum() / segment_pivot['count'].sum()

In [ ]:
iceCream = PreferenceModeling.loadVeliborIceCreamPreferenceTestData()

In [ ]:
result = solveModel(iceCream["v"], iceCream["assortments"], iceCream["stores"], iceCream["products"], num_start_cols = 2)

#### Old

In [ ]:
jda = pd.read_csv(path + 'JDA_Cloud_2017.csv')
upc = pd.read_csv(path + 'packages.csv')

In [ ]:
accts = pd.read_csv(path + 'accts_texas2.rpt', error_bad_lines=False)

In [ ]:
upc = pd.read_csv(path+'iriupc.txt', error_bad_lines=False)
iri = pd.read_csv(path +'iri1.rpt', error_bad_lines=False)

In [ ]:
accts.head()